# Predict Future Sales

We want to upgrade the current version of seaborn in this kaggle kernel to the latest one.

In [ ]:
!pip install seaborn --upgrade

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

pd.options.display.float_format = '{:.1f}'.format  # 1 decimal only for float numbers display
np.set_printoptions(suppress=True)

In [ ]:
sales = pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv", parse_dates=['date'])
test = pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")
items = pd.read_csv("../input/competitive-data-science-predict-future-sales/items.csv")
categories = pd.read_csv("../input/competitive-data-science-predict-future-sales/item_categories.csv")
shops = pd.read_csv("../input/competitive-data-science-predict-future-sales/shops.csv")

In [ ]:
sales.head()

We make a join to join sales_train with shops, items and item categories DataFrames so we can get all the data we are looking for in the same DataFrame. 

In [ ]:
sales_train = sales.join(other=shops, on="shop_id", how="inner", rsuffix="_").join(items, on="item_id", how="inner", rsuffix="_").join(categories, on="item_category_id", how="inner", rsuffix="_")
sales_train.drop(['shop_id_', 'item_id_', 'item_category_id_'], axis=1, inplace=True)

# EDA

In [ ]:
sales_train.head()

In [ ]:
sales_train.info()

In [ ]:
sales_train.describe()

In [ ]:
len(sales_train[sales_train['item_cnt_day'] < 0])

We can observe that some item_cnt_day values are negative. It seems that when an item is returned to the shop by a customer, a new row is created with a negative count to compensate for adding the item count when the customer first bought it. For the moment, we will just remove those rows from the DataFrame.

In [ ]:
sales_train = sales_train[(sales_train['item_cnt_day'] > 0) & (sales_train['item_price'] > 0)]
sales_train.isna().sum()

We want to remove item_cnt_day outliers from our data. We inspect the scatter plot and print somes percentiles to decide which threshold we shoud use to remove data.  

In [ ]:
plt.scatter(sales_train.index, sales_train['item_cnt_day'])
plt.show()

In [ ]:
p = 99
percentile = np.percentile(sales_train['item_cnt_day'], p)
print(f"The item_cnt_day {p}th percentile is equal to {percentile}")

It means that 99% of item_cnt_day values are less or equal to 5 (so lies between 0 and 5)

In [ ]:
p = 99.9
percentile = np.percentile(sales_train['item_cnt_day'], p)
print(f"The item_cnt_day {p}th percentile is equal to {percentile}")

In [ ]:
outliers_num = len(sales_train[sales_train['item_cnt_day'] > percentile])
sales_train = sales_train[sales_train['item_cnt_day'] < percentile]
print(f"We removed {outliers_num} outliers from the data")

In [ ]:
print(sales_train['item_id'].nunique())
print(sales_train['shop_id'].nunique())
print(len(test))

There is potentially 21807 x 60 (shop_id, item_id) combinations so 1 308 420 different combinations and that's pretty huge. Hopefully, we only need to forecast the combinations in the test file (214 200 combinations) and we also have to keep in mind that not all items are in all shops. 

In [ ]:
print(f"Min date from train set: {sales_train['date'].min().date()}")
print(f"Max date from train set: {sales_train['date'].max().date()}") 

sales_train['month'] = sales_train['date'].dt.month
sales_train['year'] = sales_train['date'].dt.year

## What are the 15 most sold items ?

In [ ]:
N = 15

items_total_sold = sales_train.groupby('item_id').sum()
items_total_sold.reset_index(inplace=True)

idxs = items_total_sold['item_cnt_day'].values.argsort()[::-1][0:N]
temp = items_total_sold['item_cnt_day'].to_numpy()
max_sold = [temp[idx] for idx in idxs]

item_ids = items_total_sold.loc[idxs,'item_id'].values
item_names = items.loc[item_ids, 'item_name']

fig, ax = plt.subplots(figsize=(16,12))
barplot = sns.barplot(x=max_sold, y=item_names)
for p in barplot.patches:
    barplot.text(p.get_width(), p.get_y()+0.55*p.get_height(),
                 '{:1.0f}'.format(p.get_width()),
                 ha='center', va='center')
barplot.set(xlabel="Number of items sold", ylabel="Items", title=f"Top {N} items sold")
plt.show()

We can observe that the most sold item is nothing but "Branded package T-shirt 1C Interest white (34 * 42)" (using google traduction of course, I can't understand russian yet). It makes sense, white t-shirt is a very basic cloth whether for men or women that everyone should have in their wardrobe. The rest of most sold items are video games such as Diablo III, a Massive Online Multiplayer game, Grand Theft Auto V (on 3 different platforms : PS3, Xbox360, PC) a famous RPG, Battlefied, an Online multiplayer game, and many other popular games.  
1C is one of the largest video game development, publishing and distribution studios in Russia so it's not surprising that almost only video games are among the highest ranked. 

## Shops ranking according to revenue

In [ ]:
sales_train["transaction_price"] = sales_train["item_cnt_day"] * sales_train["item_price"]
total_revenue_shops = sales_train.groupby("shop_id").agg({'transaction_price': ['sum']})
total_shops = total_revenue_shops['transaction_price']['sum'].sort_values(ascending=False)
index = total_shops.index

sns.set_theme(context="notebook", style="whitegrid", font_scale=1.3)

fig, ax = plt.subplots(figsize=(20,10))
barplot = sns.barplot(x=index, y=total_shops, order=index)
barplot.set(xlabel="Shop id", ylabel="Total revenues", title="Shops's rank in term of revenues")
plt.ticklabel_format(style='plain', axis='y')
plt.show()

## How sales behave globally ?

In [ ]:
avg_total_sales_day = sales_train.groupby('date').agg({'item_cnt_day': ['sum']}).mean().values[0]
print(f"In average, total sales per day is about {round(avg_total_sales_day)} items sold.")

In [ ]:
total_sales_day = sales_train.groupby('date').agg({'item_cnt_day': ['sum']})['item_cnt_day']['sum']

fig, ax = plt.subplots(figsize=(20,10))
lineplot = sns.lineplot(x=total_sales_day.index, y=total_sales_day)
lineplot.set(xlabel="Date", ylabel="Total sales", title="Total day sales through time among all shops")
plt.show()

We can notice that there is a seasonal pattern in total sales that seems to repeat each year. Indeed, there is a huge increase during november until late december of total sales (sales peaks). We can interpret this phenomenon as the increase of consumer purchases for Christmas period and New Year celebration that could also be strengthen by discounts during this period. The highest peaks match with days just before New Year celebration which are in terms of total sales, approximatively 4 times higher than average.

In [ ]:
avg_total_sales_month = sales_train.groupby(['year', 'month']).agg({'item_cnt_day': ['sum']}).mean().values[0]
print(f"In average, total sales per month is about {round(avg_total_sales_month)} items sold.")

In [ ]:
y = sales_train.groupby(['year','month']).agg({'item_cnt_day': ['sum']})['item_cnt_day']['sum'].values

date = []
years=['2013','2014','2015']
for year in years:
    for month in range(1,13):
        if month<10:
            date.append(year+'-0'+str(month))
        else:
            date.append(year+'-'+str(month))

fig, ax = plt.subplots(figsize=(20,10))
lineplot = sns.lineplot(x=date, y=y)
lineplot.set(xlabel="Date", ylabel="Total sales", title="Total month sales through time among all shops")
lineplot.set_xticklabels(date, rotation=50)
plt.show()

We can confirm the trends described earlier in this plot and also say that there is a continued decline in total month sales over time. For instance, total month sales were about 120 000 items sold in january 2013, then went down to near 100 000 in january 2014 and finally only 80 000 in january 2015. A reason that could explain the general decrease is the fact that the list of shops and products slightly changes every month as written in the "Data" section of this kaggle project. Maybe some high-performance shops have left the list of shops or/and some popular items have left the list of items.   

Moreover, it seems like there is a huge drop in sales since october 2015 (going from 68 000 to 28 000 next month, almost a division of sales by a factor of 2,5 !). It's pretty unusual as this period is normally very conductive to sales as Christmas Eve approaches. That happened because of missing data between 2015-11-10 and 2015-12-01 (so 20 missing days of data) as suggested by the day sales plot. Maybe an error during concatenation of data occured when building the dataset. 

To conclude, the trend in sales is clearly downwards. 

In [ ]:
sales_train.groupby('date').agg({'item_cnt_day': ['sum']}).tail(12)

## What are the 10 most popular categories ?

In [ ]:
N = 10

total_categories_sold = sales_train.groupby('item_category_id').sum()
items_sold_by_category = total_categories_sold['item_cnt_day'].nlargest(n=N)
idxs = items_sold_by_category.index

categories_names = categories.loc[idxs, 'item_category_name']

fig, ax = plt.subplots(figsize=(16,8))
barplot = sns.barplot(x=categories_names, y=items_sold_by_category)
barplot.set(xlabel="Category name", ylabel="Total items sold", title=f"Top {N} most popular categories")
for label in barplot.get_xticklabels():
    label.set_rotation(50)
for p in barplot.patches:
    barplot.annotate(format(p.get_height(), '1.0f'), 
                    (p.get_x() + p.get_width() / 2., p.get_height()), 
                     ha='center', va='center', 
                     xytext=(0, 7), 
                     textcoords='offset points')
plt.ticklabel_format(style='plain', axis='y')
plt.show()

We can say from this plot that movies popularity is way ahead from the others categories with 634 171 items sold in total. DVD quality (720x576 pixels resolution) is less good than Blu-Ray quality (1920x1080 pixels resolution) but it's still the most prefered category. It could be explained by multiple reasons including the price difference between those two categories (DVD is cheaper than Blu-Ray), the fact that Blu-Ray doesn't have a movie catalog as large as the DVDs (especially old titles) and also because there is still a significant part of russian families that don't have a full HD compatible TV to be able to play Blu-Ray movies.  
Then we find PC games (standard and additional editions) as well as console games (PS3, XBOX 360, PS4 but not Xbox One, Sony seems to have one the latest generation of game console) and also music CD and albums. 

# Predictions

## Univariate LSTM Model

Let's build an univariate time series predictive model that will be able to forecast the next month sales based on month sales sequences. 
We group data by month, shop_id, item_id and aggregate sales by month using sum function. We then use pivot table to present data in an interpretable way (shop_id, item_id as index and month as columns). 

In [ ]:
data_sales = sales_train.groupby(['date_block_num', 'shop_id', 'item_id'], as_index=False)['item_cnt_day'].sum()
data_sales.rename(columns={'item_cnt_day': 'item_cnt_month'}, inplace=True)
data_sales = pd.pivot_table(data_sales, index=['shop_id','item_id'],columns=['date_block_num'], fill_value=0)

In [ ]:
data_sales

We must now merge the test dataset with our data DataFrame because we only need to predict the (shop_id,item_id) combinations contained in the test dataset.

In [ ]:
df_sales = data_sales.merge(test, how="right", on=['shop_id','item_id'])
per = round(df_sales.isna().sum()[2] / len(df_sales), 2)
df_sales

In [ ]:
print(f"The percentage of (shop_id, item_id) combinations with NaN values is {per}")

That's interesting, 48% of rows of our merged DataFrame are (shop_id, item_id) combinations which were not find in the data DataFrame (training set) so their item_cnt_month cells are full empty with NaN values. We must fill them with 0 as we can't do anything better right now. Intuitively, putting 0 sales for every (item_id shop_id) combination that isn't in training data means that every item not listed in a shop hasn't been sold.  
We also delete shop_id, item_id and ID columns, they are not useful for our model. Indeed we want to create an LSTM model which will be feed on time series data, i.e. monthly sales for each combination. 

In [ ]:
df_sales.fillna(0, inplace=True)
df_sales.drop(['shop_id', 'item_id', 'ID'], axis=1, inplace=True)
df_sales.head()

We prepare our train/validation/test sets properly. Our splitting strategy is the following :
1. We train our time series model from 0 to 32 date_block_num (from January 2013 to September 2015) and try to predict 33 (October 2015 sales) in order to learn patterns from data.
2. To better evaluate the model's performance (at each epoch), we split our data between train and validation sets (90% training and 10% validation). We use validation MSE (Mean Square Error) metric to monitor performance during training. 
3. We make predictions of November 2015 sales using our trained LSTM model with test set from 1 to 33 date_block_num (from February 2013 to October 2015). 

Each sets must have the same dimensions (so same number of timesteps as we want to build an univariate model here, i.e features=1). 

In [ ]:
from sklearn.model_selection import train_test_split

X, y = df_sales.drop(labels=[('item_cnt_month',33)], axis=1).values, df_sales.values[:,-1]
X_test = df_sales.drop(labels=[('item_cnt_month',0)], axis=1).values

# We need to reshape the data accordingly to Keras input shape (bach_size, timesteps, features)
X = X.reshape(X.shape[0], X.shape[1], 1)
y = y.reshape(y.shape[0], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.10, shuffle=True, random_state=0)

print("X_train shape : ", X_train.shape)
print("y_train shape : ", y_train.shape)
print("X_val shape : ", X_val.shape)
print("y_val shape : ", y_val.shape)
print("X_test shape : ", X_test.shape)

In [ ]:
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping

tf.keras.backend.set_floatx('float64')
early_stopping = EarlyStopping(monitor='val_loss', patience=2, mode='auto')

def plot_train_val_curves(hist):
    fig, (ax1, ax2) = plt.subplots(2, figsize=(15,8), sharex=True)
    fig.suptitle('Loss and MSE train and validation curves')
    ax1.plot(hist.history['loss'], label='train')
    ax1.plot(hist.history['val_loss'], label='validation')
    ax1.set_xlabel("Epochs")
    ax1.set_ylabel("Loss")
    ax1.legend()
    ax2.plot(hist.history['mean_squared_error'], label='train')
    ax2.plot(hist.history['val_mean_squared_error'], label='validation')
    ax2.set_xlabel("Epochs")
    ax2.set_ylabel("MSE")
    ax2.legend()
    plt.show()

def LSTM_model(shape, units=[64,64], dropout=0.3):
    model = Sequential()
    model.add(Input(shape=shape, dtype='float64'))
    model.add(LSTM(units=units[0], activation='tanh', return_sequences=True))
    model.add(Dropout(dropout))
    model.add(LSTM(units=units[1], activation='tanh'))
    model.add(Dropout(dropout))
    model.add(Dense(1, activation=None, kernel_initializer=tf.initializers.zeros()))
    return model

In [ ]:
lstm_model = LSTM_model(shape=(X_train.shape[1],X_train.shape[2]), units=[128,128], dropout=0.4)
lstm_model.compile(optimizer="Adam", loss="mse", metrics=['mean_squared_error'])
lstm_model.summary()
history = lstm_model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=4096, epochs=10, callbacks=[early_stopping])
lstm_model.save("lstm_model", overwrite=True)

In [ ]:
plot_train_val_curves(history)

Why is validation loss and MSE a little bit lower than training loss and MSE ? At first look, that's counter intuitive but it is actually normal in this case when we think about it. Our LSTM architecture includes 2 dropout layers that are used during training loop but are NOT in validation loop (Keras default behavior). Regularization methods often sacrifice training accuracy to improve validation/testing accuracy. This fact can explain by itself why it gives more accurate predictions on validation data and so a lower loss. Another reason is that training loss is measured during each epoch while validation loss is measured after each epoch. Training loss is continually reported over the course of an entire epoch, however, validation metrics are computed over the validation set only once the current training epoch is completed. A third reason would be that training and validation distributions are different but that's not our case as we shuffled our data before splitting it. 

We predict November 2015 sales using our trained model. We replace negative predictions by zero values (no sales) and large values by 20, in another words, we clip predictions between 0 and 20. As we have seen at the beginning of this notebook, the distribution of sales lies in this area (outliers are above). 

In [ ]:
predictions = lstm_model.predict(X_test)
predictions = predictions.clip(0,20)

In [ ]:
per = round(predictions[predictions > 1].sum() / len(predictions), 2) * 100
print(f"{per} % of sales predictions are higher than 1")

Finally, we export submission DataFrame in csv file without writting the default index in it (ID column is already an index). 

In [ ]:
test_submission = pd.DataFrame({'ID': test['ID'], 'item_cnt_month': predictions.flatten()})
test_submission.to_csv('test_submission.csv', index=False) 

## Multivariate LSTM model

This time, we are going to build a multivariate LSTM model by adding another important feature : item_price. We will aggregate monthly item_price using mean function. This feature will be a good indicator of the price variation and will help as a predictor to get more accurate forecast sales. We will use the same data processing stages that we used for the last model. We will get at the end 2 features for each timestep (1 timestep = 1 month) in our time series data.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# We scale item_price in our train and test set. This will stabilize training and make it faster.
scaler = MinMaxScaler()
sales_train['item_price'] = scaler.fit_transform(sales_train['item_price'].values.reshape(-1,1))

In [ ]:
data_prices = sales_train.groupby(['date_block_num', 'shop_id', 'item_id'], as_index=False)['item_price'].mean()
data_prices.rename(columns={'item_price': 'item_price_mean'}, inplace=True)
data_prices = pd.pivot_table(data_prices, index=['shop_id','item_id'], columns=['date_block_num'], fill_value=0)
data_prices

In [ ]:
df_prices = data_prices.merge(test, how="right", on=['shop_id','item_id']).fillna(0)
df_prices.drop(['shop_id', 'item_id', 'ID'], axis=1, inplace=True)

# get each feature for training data and labels 
X_sales = df_sales.drop(labels=[('item_cnt_month',33)], axis=1).values
X_prices = df_prices.drop(labels=[('item_price_mean',33)], axis=1).values
y = df_sales.values[:,-1]

# reshape arrays
X_sales = X_sales.reshape(X_sales.shape[0], X_sales.shape[1], 1)
X_prices = X_prices.reshape(X_prices.shape[0], X_prices.shape[1], 1)
# combine sales and prices in the same numpy array
X = np.append(X_sales, X_prices, axis=2)
y = y.reshape(y.shape[0], 1)

# construct test data (X_test)
X_sales_test = df_sales.drop(labels=[('item_cnt_month',0)], axis=1).values
X_prices_test = df_prices.drop(labels=[('item_price_mean',0)], axis=1).values
X_sales_test = X_sales_test.reshape(X_sales_test.shape[0], X_sales_test.shape[1], 1)
X_prices_test = X_prices_test.reshape(X_prices_test.shape[0], X_prices_test.shape[1], 1)
X_test = np.append(X_sales_test, X_prices_test, axis=2)

# split data in train and validation sets 
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.10, shuffle=True, random_state=0)

print("X shape : ", X.shape)
print("y shape : ", y.shape)
print("X_train shape : ", X_train.shape)
print("y_train shape : ", y_train.shape)
print("X_val shape : ", X_val.shape)
print("y_val shape : ", y_val.shape)
print("X_test shape : ", X_test.shape)

In [ ]:
mult_lstm_model = LSTM_model(shape=(X_train.shape[1],X_train.shape[2]), units=[128,128], dropout=0.4)
mult_lstm_model.compile(optimizer="Adam", loss="mse", metrics=['mean_squared_error'])
mult_lstm_model.summary()
history = mult_lstm_model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=4096, epochs=10, callbacks=[early_stopping])
mult_lstm_model.save("mult_lstm_model", overwrite=True)

In [ ]:
plot_train_val_curves(history)

In [ ]:
predictions = mult_lstm_model.predict(X_test)
predictions = predictions.clip(0,20)
test_submission = pd.DataFrame({'ID': test['ID'], 'item_cnt_month': predictions.flatten()})
test_submission.to_csv('test_submission.csv', index=False) 